In [1]:
%pip install python-dotenv docx2txt langchain_community langchain-text-splitters langchain-chroma google-generativeai langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)


In [4]:
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") 


In [5]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [12]:
query = '제 55조를 근거로 5천만원의 거주자에 대한 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query)
retrieved_docs

[Document(id='b886ec4c-2535-4516-89a8-566022dad689', metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준        | 세율                                                      |\n\n|----------------------|-----------------------------------------------------------|\n\n| 1,400만원 이하        | 과세표준의 6퍼센트                                            |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)                     |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)                    |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)                   |\n\n| 1억5천만원 초과 3억원 이하   | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)                  |\n\n| 3억원 초과 5억원 이하     | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)                     |\n\n| 5억원 초과 10억원 이하    | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)   

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash', temperature=0.9)

In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\82104\VscodeProjects\LLM\inflearn-llm-application\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [16]:
ai_message = qa_chain({"query": query})

C:\Users\82104\AppData\Local\Temp\ipykernel_15136\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [17]:
ai_message

{'query': '제 55조를 근거로 5천만원의 거주자에 대한 소득세는 얼마인가요?',
 'result': '5천만원의 거주자에 대한 소득세는 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)로 계산됩니다. 5천만원은 종합소득 과세표준 5,000만원 초과 8,800만원 이하 구간에 해당됩니다.'}